In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
question = '''为了还原用户UI操作流程，需要识别哪些UI操作是相同的，哪些是不同的。比如，提交报销单，两个人填写的报销单可能只有姓名、部门、时间这些内容是相似的，
其他的比如出差事由、时间范围等等相差非常大，在页面上看起来也很不一样。但是，它们同属于一个业务操作，应该判断相同操作，即使对应的窗口/应用从视觉上和内容上来看相差非常大。
仅用图片相似性无法解决这个问题，因为文字内容变化难以检测；仅用文字比较也不行，因为内容相差很大的UI操作可能属于同一个业务操作。
'''

solution = '''使用CV算法识别窗口/应用的区域，同一个窗口的区域分布应该是完全一样的。另外，再提取一两个关键标签，代表当前操作的内容。只要窗口区域相似，
关键标签相同，就可以判断两个UI操作属于同一个业务操作。识别窗口区域的方法是canny，通过边框识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的。
提交关键标签，是指使用OCR识别窗口中的所有文字，然后提取关键位置的文字，比如窗口标题栏中的文字。两者结合就可以较有效地区分不同窗口，进而判断窗口的相似性。
'''

In [4]:
template_background = '''你是一个专业的流程挖掘领域的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的背景介绍部分。
先回答以下问题：待解决的问题属于哪个技术领域？在这个领域中，现有软件或者技术存在什么样的问题或者局限性？发明点要解决的具体问题是什么？
根据上述问题的答案，生成背景介绍。
要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的背景介绍：'''

prompt_background = PromptTemplate(input_variables=['question', 'solution'], template=template_background)
background_chain = LLMChain(llm=llm, prompt=prompt_background, output_key='background')

In [5]:
template_claims = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据以下要解决的问题及解决方案，起草专利的权利要求部分。
在设计专利的权利要求时要一步步来，先判断解决方案的创新性和价值所在，然后再根据创新点来设计对应的权利要求。针对要解决的问题，先将解决方案重新描述为多个步骤，再判断每一个步骤
是否有必要存在，若是没有必要，则可以删除；对于必要存在的步骤，再考虑它是否具备创新性。若是不具备，则更新对应的步骤。只要其中一个步骤具备创新性即可。
可使用以下问题来引导生成过程：解决方案的基本思路是什么？它的功能和特征是什么？它如何解决上述问题？有哪些优点或者创新？

要解决的问题: {question}
提供的解决方案: {solution}
请生成这个专利发明点的权利要求：'''

prompt_claims = PromptTemplate(input_variables=['question', 'solution'], template=template_claims)
claims_chain = LLMChain(llm=llm, prompt=prompt_claims, output_key='claims')

In [6]:
template_explanation = '''你是一个专业的流程挖掘和任务挖掘方面的专利代理，负责专利发明点设计和专利文档起草。请根据解决的问题、解决方案及权利要求，
扩充权利要求部分的内容。扩充的方法是，先设计一个具体的应用场景，将权利要求转换为具体的实现步骤，并结合应用场景进行描述每一步的输入输出，以及中间的处理过程。
若是不同权利要求对应的操作步骤是相同的，则只需要在第一次给出描述，后面的则省略。
可使用这些问题来引导生成过程：它的主要算法或者技术是什么？需要什么样的软硬件运行环境？是否有多种实现方式？每种实现方式最简单的形式是什么样的？

要解决的问题: {question}
提供的解决方案: {solution}
独立权利要求: {claims}

请将权利要求扩展为具体的实现步骤，并给出相应的示例说明：'''

prompt_explanation = PromptTemplate(input_variables=['question', 'solution', 'claims'], template=template_explanation)
explanation_chain = LLMChain(llm=llm, prompt=prompt_explanation, output_key='explanation')

In [7]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[background_chain, claims_chain, explanation_chain],
    input_variables=["question", "solution"],
    # Here we return multiple variables
    output_variables=["background", "claims", "explanation"],
    verbose=True)

In [8]:
result = overall_chain({'question': question, 'solution': solution})



> Entering new SequentialChain chain...

> Finished chain.


In [9]:
print(result['background'])

本发明点涉及流程挖掘领域，具体地说是解决识别用户UI操作流程中相同和不同操作的问题。现有软件或技术无法有效区分相似但实际上不同的UI操作，这会导致流程分析的不准确甚至错误。本发明点的目的是识别哪些UI操作是相同的，哪些是不同的，从而更准确地还原用户UI操作流程。本发明点的核心解决方案是使用CV算法识别窗口/应用的区域，然后提取关键标签，代表当前操作的内容。通过窗口区域的相似性和关键标签的相同性，可以判断两个UI操作是否属于同一个业务操作。因此，本发明点提供了一种高效、准确的方法来区分不同的UI操作，从而提高流程挖掘的准确度和可靠性。


In [10]:
print(result['claims'])

1. 一种用于还原用户UI操作流程的方法，包括以下步骤：
    a. 识别窗口/应用的区域；
    b. 提取关键标签，代表当前操作的内容；
    c. 判断两个UI操作属于同一个业务操作，当且仅当它们的窗口区域相似，关键标签相同；
    其中，步骤a使用canny算法识别窗口的主要布局，同一个应用内只有有限的窗口，每个窗口的布局是很少变化的；步骤b使用OCR识别窗口中的所有文字，然后提取关键位置的文字，比如窗口标题栏中的文字。

2. 根据权利要求1所述方法，通过计算窗口区域和关键标签的相似性，判断两个UI操作是否属于同一个业务操作的系统。

3. 根据权利要求1所述方法，通过对窗口/应用的区域和关键标签进行训练，提高判断两个UI操作是否属于同一个业务操作的准确性的系统。

4. 根据权利要求1所述方法，通过记录用户UI操作流程的方式，提供用户UI操作流程还原的服务。

5. 根据权利要求1所述方法，通过对不同窗口/应用的区域进行分类管理，实现对UI操作流程的自动化识别、分类和分析的系统。


In [11]:
print(result['explanation'])

6. 根据权利要求1所述方法，进一步细化步骤a，包括以下步骤：
    a1. 对窗口截图进行预处理，如灰度化、二值化、去噪等；
    a2. 使用canny算法提取窗口边缘；
    a3. 对边缘进行霍夫线变换，得到窗口四条边的位置信息；
    a4. 根据窗口四条边的位置信息，确定窗口区域。

示例说明：用户需要提交一个报销申请，打开报销申请系统的页面。该页面有不同的窗口，如登录窗口、填写报销单窗口等。对于填写报销单窗口，其窗口区域应该是相同的，包括报销单表格、提交按钮等。通过预处理、边缘提取和霍夫线变换，可以确定窗口的四个边界，从而确定窗口区域。

7. 根据权利要求1所述方法，进一步细化步骤b，包括以下步骤：
    b1. 对窗口截图进行预处理，如灰度化、二值化、去噪等；
    b2. 使用OCR技术识别窗口中的所有文字；
    b3. 根据预设规则，提取关键位置的文字；
    b4. 将提取的文字作为关键标签。

示例说明：用户在填写报销单时，需要在窗口中输入报销信息，如报销单编号、报销金额等。通过OCR技术，可以将所有文字识别出来。然后根据预设规则，如只提取窗口标题栏中的文字，将其作为关键标签。

8. 根据权利要求2所述方法，进一步细化步骤c，包括以下步骤：
    c1. 对窗口区域进行相似度计算，得到相似度；
    c2. 对关键标签进行相似度计算，得到相似度；
    c3. 根据预设阈值，判断两个UI操作是否属于同一个业务操作。

示例说明：用户A和用户B都需要提交报销申请，但是填写的报销单信息不同。通过计算窗口区域和关键标签的相似度，可以判断两个UI操作是否属于同一个业务操作。当窗口区域相似度和关键标签相似度都超过预设阈值时，说明两个UI操作属于同一个业务操作。

9. 根据权利要求3所述方法，进一步细化步骤c，包括以下步骤：
    c4. 对窗口区域和关键标签进行训练，建立模型；
    c5. 使用模型进行预测，提高判断准确度。

示例说明：为了提高判断准确度，可以对窗口区域和关键标签进行训练，建立模型。通过模型进行预测，可以提高判断准确度。

10. 根据权利要求4所述方法，包括以下步骤：
    d1. 记录用户UI操作流程；
    d2. 根据记录的UI操作流程，还原用户操作流程。

示例说明：用户在填写报销单时

In [12]:
with open('patent-output3.txt', 'w') as f:
    f.write('background: ' + result['background'])
    f.write('\n\nclaims: ' + result['claims'])
    f.write('\n\nexplanation: ' + result['explanation'])